In [1]:
import gradio as gr
from dotenv import load_dotenv
import os
import google.generativeai as genai

# 환경 변수 로드
load_dotenv()

# API 키 설정
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

def recommend_dishes(ingredients):
    prompt = f"주어진 재료들로 어떤 음식을 만들 수 있는지 한 줄로만 콤마(',')로 분리해서 출력해. `{ingredients}`"
    try:
        model = genai.GenerativeModel("gemini-1.5-pro")
        response = model.generate_content(prompt)
        # response = genai.generate_text(prompt=prompt, model="models/text-bison-001")
        # if response.result:
        #     # dishes = response.result.split("\n")  # 음식 리스트를 줄바꿈으로 분리
        #     # print(f"Generated dishes: {dishes}")  # 디버깅을 위한 출력
        #     # return dishes
        response_text = response.candidates[0].content.parts[0].text

        # print(response_text)
        # else:
        #     print("No result from the API")
        #     return []
        # 달걀, 파, 양파, 닭고기
        print( [i.strip() for i in response_text.split(',')])
        return [i.strip() for i in response_text.split(',')]
    except Exception as e:
        print(f"Error in recommend_dishes: {e}")  # 에러 메시지를 출력하여 디버깅에 도움을 줍니다.
        return []

def get_recipe(dish):
    prompt = f"`{dish}`을 어떻게 조리하는지 조리 과정을 설명해."
    try:
        model = genai.GenerativeModel("gemini-1.5-pro")
        response = model.generate_content(prompt)
        if response:
            return response.candidates[0].content.parts[0].text
        else:
            print("No result from the API for the recipe")
            return "레시피를 가져오는 데 오류가 발생했습니다."
    except Exception as e:
        print(f"Error in get_recipe: {e}")  # 에러 메시지를 출력하여 디버깅에 도움을 줍니다.
        return "레시피를 가져오는 데 오류가 발생했습니다."

# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    gr.Markdown("### 음식 재료를 입력하면 만들 수 있는 음식을 추천해드립니다.")

    ingredients = gr.Textbox(label="음식 재료 (콤마로 구분)")
    output_dishes = gr.Dropdown(label="추천된 음식 리스트", choices=[])
    output_recipe = gr.Textbox(label="요리 레시피", interactive=False)

    recommend_button = gr.Button("음식 추천받기")
    get_recipe_button = gr.Button("레시피 확인")

    def update_dishes(ingredients):
        dishes = recommend_dishes(ingredients)
        if dishes:  # 리스트가 비어있지 않다면 업데이트
            output_dishes.choices = dishes
            output_dishes.value = dishes[0] if dishes else ""
        else:  # 비어있다면 초기화
            output_dishes.choices = []
            output_dishes.value = ""
        return gr.Dropdown(choices=dishes, interactive=True)
        
    # 음식 추천 버튼 클릭 시
    recommend_button.click(fn=update_dishes, inputs=ingredients, outputs=output_dishes)
    # 레시피 확인 버튼 클릭 시
    get_recipe_button.click(fn=get_recipe, inputs=output_dishes, outputs=output_recipe)

demo.launch()


Running on local URL:  http://127.0.0.1:7896

To create a public link, set `share=True` in `launch()`.
